In [ ]:
#导入Raspbot驱动库及相关库 Import the Raspbot library and related libraries
from Raspbot_Lib import Raspbot,LightShow
from ipywidgets import interact
import ipywidgets as widgets
import time
bot = Raspbot()

### 程序1-@按键切换颜色 Program 1 - @ button to switch colors

按下KEY1键在七种颜色之间循环 Press KEY1 to cycle through the seven colors

In [ ]:
colors = [0, 1, 2, 3, 4, 5, 6, 7]  # 红色、绿色、蓝色、黄色、紫色、青色、白色、关闭 。Red, Green, Blue, Yellow, Purple, Cyan, White, Off
current_color = 0  # 当前颜色索引 当前颜色索引
key_down = False
while True:
        data = bot.read_data_array(0x0d, 1)
        state = data[0]

        # 只有当按键从未按下变为按下时才切换颜色 Switch color only when the key changes from unpressed to pressed
        if state == 1 and not key_down:
            current_color = (current_color + 1) % len(colors)
            if(current_color==7):
                 bot.Ctrl_WQ2812_ALL(0,0)
            else :
                 bot.Ctrl_WQ2812_ALL(1, colors[current_color])

            key_down = True  

        if state == 0 and key_down:
            key_down = False  

### 程序2-@灯条测试 Program 2 - @Light bar test

按钮widgets.Button参数解释说明：
description：表示显示在按钮上的名称。
Button widgets.Button parameter explanation: description: indicates the name displayed on the button.

容器widgets.Box参数解释说明：
widgets.Box能把多个控件打包在一起显示，下面是把几个按钮打包在一起显示。
Container widgets.Box parameter explanation:
widgets.Box can package multiple controls together for display. The following is a package of several buttons for display.

In [ ]:
# 按钮事件处理，按下不同按钮显示不同颜色
# Button event processing, press different buttons to display different colors
def btn_click(info):
    str_id = info.description
    print('Button %s clicked!' % str_id)
    if str_id == "Red":
        bot.Ctrl_WQ2812_ALL(1,0)#红色
    elif str_id == "Green":
        bot.Ctrl_WQ2812_ALL(1,1)#绿色
    elif str_id == "Blue":
        bot.Ctrl_WQ2812_ALL(1,2)#蓝色
    elif str_id == "Yellow":
        bot.Ctrl_WQ2812_ALL(1,3)#黄色
    elif str_id == "Purple":
        bot.Ctrl_WQ2812_ALL(1,4)#紫色
    elif str_id == "Indigo":
        bot.Ctrl_WQ2812_ALL(1,5)#青色
    elif str_id == "White":
        bot.Ctrl_WQ2812_ALL(1,6)#白色    
    elif str_id == "OFF":
        bot.Ctrl_WQ2812_ALL(0,0)
        R(value=0)


# 创建一个按钮，功能：显示红色
# Create a button that displays red
btn_red = widgets.Button(description = "Red")    
btn_red.on_click(btn_click)

# 创建一个按钮，功能：显示绿色
# Create a button that displays green
btn_green = widgets.Button(description = "Green")    
btn_green.on_click(btn_click)

# 创建一个按钮，功能：显示蓝色
# Create a button that displays blue
btn_blue = widgets.Button(description = "Blue")    
btn_blue.on_click(btn_click)

# 创建一个按钮，功能：显示黄色
# Create a button that displays yellow
btn_yellow = widgets.Button(description = "Yellow")    
btn_yellow.on_click(btn_click)

# 创建一个按钮，功能：显示紫色
# Create a button that displays purple
btn_purple = widgets.Button(description = "Purple")    
btn_purple.on_click(btn_click)

# 创建一个按钮，功能：显示青色
# Create a button that displays indigo 
btn_indigo = widgets.Button(description = "Indigo")    
btn_indigo.on_click(btn_click)

# 创建一个按钮，功能：显示白色
# Create a button that displays white
btn_white = widgets.Button(description = "White")    
btn_white.on_click(btn_click)

# 创建一个按钮，功能：关闭显示
# Create a button that displays off
btn_off = widgets.Button(description = "OFF")    
btn_off.on_click(btn_click)

box = widgets.Box([btn_red, btn_green, btn_blue, btn_yellow])
display(box)
box1 = widgets.Box([btn_purple, btn_indigo, btn_white, btn_off])
display(box1)

def set_rgb(R, G, B):
    bot.Ctrl_WQ2812_brightness_ALL(R, G, B)
    return R, G, B

# 创建三个个滑块来控制RGB值  Create three sliders to control the RGB values
interact(set_rgb, \
         R=widgets.IntSlider(min=0,max=255,step=1,value=0), \
         G=widgets.IntSlider(min=0,max=255,step=1,value=0), \
         B=widgets.IntSlider(min=0,max=255,step=1,value=0));
         


### 程序3-@灯效测试 Program 3-@Lighting effect test
下拉菜单控件Dropdown参数解释：
options：是一个列表list，表示可以选取的选项，其元素既可以是简单的str值，也可以是形如(label, value)的键值对；
label：是当前选项在页面上显示的名称
value：是当前选项在控件内部使用的值，如果没手动设置，默认=label
index：是当前选项在options中的索引值，自动分配，从0开始按顺序增加
如果options元素不是键值对而是str，则label与value相等
如果在初始化时同时指定value、label与index，则以最后指定的值为准

Parameter Description: Dropdown menu control  
Options: a list of options that can be selected. The elements can be simple STR values or key-value pairs such as (label, value).  
Label: is the name of the current option displayed on the page  
Value: Is the value that the current option uses inside the control. If this is not set manually, the default value is =label  
Index: indicates the index value of the current option in options. It is automatically allocated and increases from 0  
If the options element is a STR instead of a key-value pair, then label is equal to value  
If value, label, and index are specified at the same time during initialization, the last specified value prevails  


```shell

In [ ]:
effect_Dropdown = widgets.Dropdown(options=["OFF", "Gradient Light", "Flowing Light", "Running Light", "Sparkling Light"], index=0)
lights = LightShow()
global g_effect
g_effect = 0
g_duration = 5
g_speed = 0.01
# 处理下拉菜单选择的内容，并执行相应的特效。
def effect_handle(_):
    print("Selected index:{}, value:{}".format(effect_Dropdown.index, effect_Dropdown.value))

    str_effect = effect_Dropdown.label
    if str_effect == "OFF":
        g_effect=0
        lights.turn_off_all_lights()

    # elif str_effect == "Breathing Effect":
    #     g_effect='breathing'
    #     lights.execute_effect(g_effect,g_duration,g_speed,0)
    #     effect_Dropdown.label="OFF"
        
    elif str_effect == "Gradient Light":
        g_effect='gradient'
        lights.execute_effect(g_effect,g_duration,g_speed,0)
        print("finished")
        effect_Dropdown.label="OFF"
        
    elif str_effect == "Flowing Light":
        g_effect='river'
        lights.execute_effect(g_effect,g_duration,g_speed,0)
        print("finished")
        effect_Dropdown.label="OFF"

    elif str_effect == "Running Light":
        g_effect='random_running'
        lights.execute_effect(g_effect,g_duration,g_speed,0)
        print("finished")
        effect_Dropdown.label="OFF"

    elif str_effect == "Sparkling Light":
        g_effect='starlight'
        lights.execute_effect(g_effect,g_duration,g_speed,0)
        print("finished")
        effect_Dropdown.label="OFF"


# 创建一个下拉菜单 Create a drop-down menu
effect_Dropdown.observe(effect_handle, names="value")
display(effect_Dropdown)

# 控制灯效时长,速度 time=1为最短，20为最长,speed=0.01为最快,0.5为最慢
# Control the duration of the lighting effect, speed time=1 is the shortest, 20 is the longest
# ,speed=0.01 is the fastest, 0.5 is the slowest

def duration_handle(speed,duration):
    global g_speed,g_duration
    g_speed = speed
    g_duration = duration



# 创建一个滑块，控制灯效时长 Create a slider to control the duration of the lighting effect
interact(duration_handle, \
        speed=widgets.FloatSlider(min=0.01,max=0.2,step=0.01,value=0.01), \
        duration=widgets.IntSlider(min=1,max=20,step=1,value=5));
#Gradient Light speed 0.02
#Flowing Light speed 0.01
#Running light speed 0.1
#Sparking Lihgt speed 0.1

In [ ]:
#Breathing Effect
#参数2：默认值为5,灯效持续时间
#参数3：默认值为0.01, 灯效果速度
#参数4：默认值为0, 0红 1绿 2蓝 3黄 4紫 5青 6白
#Parameter 2: Default value is 5, duration of light effect
#Parameter 3: Default value is 0.01, speed of light effect
#Parameter 4: Default value is 0, 0 red 1 green 2 blue 3 yellow 4 purple 5 cyan 6 white
lights.execute_effect('breathing',5,0.01,0)

#### @结束释放资源  releasing resources

In [ ]:
del bot                #使用完成对象记住释放掉对象，不然下一个程序使用这个对象模块会被占用，导致无法使用